### Import

In [10]:
from sqlalchemy import create_engine
import os
import yaml
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


### Read Config

In [11]:
base = Path().resolve()
db_conf_file = os.path.normpath(os.path.join(base, "../conf/db_connection.yaml"))
with open(db_conf_file, "r", encoding="utf-8") as l_file:
    db_conf_all = yaml.safe_load(l_file)

### Create Engine

In [12]:
ora_conf = db_conf_all["oracle"]
engine = create_engine(f"oracle://{ora_conf['username']}:{ora_conf['password']}@{ora_conf['dsn']}/?encoding=UTF-8&nencoding=UTF-8")

### Condition

In [13]:
c_cource_id = "05_t1800"

### Query

In [14]:
from sqlalchemy.sql import select
from sqlalchemy.sql import join

from model.race import Race
from keibaai.model.race import RaceUma

query_statement = (
    select(
        join(Race, RaceUma, Race.id == RaceUma.race_id)
        ).where(
            Race.course_id == c_cource_id
        )
)
print(query_statement)

SELECT races.id, races.name, races.race_track_id, races.kai, races.nichi, races.race_no, races.course_id, races.grade_id, races.is_win5, races.condition, races.handicap, races.race_date, races.race_start, races.weather, races.going, races.num_of_horses, races.race_data, races.corner_order_1, races.corner_order_2, races.corner_order_3, races.corner_order_4, races.pace, race_uma.race_id, race_uma.uma_id, race_uma.result, race_uma.bracket_number, race_uma.horse_number, race_uma.gender, race_uma.age, race_uma.weight_to_carry, race_uma.jockey_id, race_uma.time, race_uma.margin, race_uma.ninki, race_uma.win_odds, race_uma.final_3_furlong, race_uma.corner_order, race_uma.trainer_id, race_uma.horse_weight, race_uma.gain_and_loss_weight, race_uma.is_excluded, race_uma.is_demoted 
FROM races JOIN race_uma ON races.id = race_uma.race_id 
WHERE races.course_id = :course_id_1


In [15]:
query_statement = (
    select(
        Race.race_track_id, Race.kai, Race.nichi, Race.race_no, Race.grade_id, Race.num_of_horses, RaceUma.result, RaceUma.bracket_number, RaceUma.horse_number, RaceUma.age, RaceUma.weight_to_carry, RaceUma.time, RaceUma.ninki, RaceUma.win_odds, RaceUma.final_3_furlong, RaceUma.horse_weight, RaceUma.gain_and_loss_weight
    ).select_from(
        join(Race, RaceUma, Race.id == RaceUma.race_id)
    ).where(
        Race.course_id == c_cource_id
    )
)
print(query_statement)

SELECT races.race_track_id, races.kai, races.nichi, races.race_no, races.grade_id, races.num_of_horses, race_uma.result, race_uma.bracket_number, race_uma.horse_number, race_uma.age, race_uma.weight_to_carry, race_uma.time, race_uma.ninki, race_uma.win_odds, race_uma.final_3_furlong, race_uma.horse_weight, race_uma.gain_and_loss_weight 
FROM races JOIN race_uma ON races.id = race_uma.race_id 
WHERE races.course_id = :course_id_1


### Read Data

In [16]:
data = pd.read_sql_query(sql=query_statement, con=engine)
data

,race_track_id,kai,nichi,race_no,grade_id,num_of_horses,result,bracket_number,horse_number,age,weight_to_carry,time,ninki,win_odds,final_3_furlong,horse_weight,gain_and_loss_weight
0,50,10,30,6,31,13,1.0,6,8,3,56.0,107.6,2,3.2,34.9,452,0.0
1,50,10,30,6,31,13,2.0,8,13,3,56.0,108.0,3,5.1,35.1,478,0.0
2,50,10,30,6,31,13,3.0,3,3,3,56.0,108.2,4,5.7,34.9,486,0.0
3,50,10,30,6,31,13,3.0,5,6,3,56.0,108.2,1,2.7,35.0,490,0.0
4,50,10,30,6,31,13,5.0,4,4,3,54.0,108.6,11,130.8,35.2,452,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5036,50,30,80,9,17,9,5.0,1,1,5,55.0,107.0,4,14.9,35.2,490,2.0
5037,50,30,80,9,17,9,6.0,8,8,4,55.0,107.5,8,142.3,34.4,422,0.0
5038,50,30,80,9,17,9,7.0,4,4,6,57.0,108.3,6,46.6,35.7,468,-2.0
5039,50,30,80,9,17,9,8.0,8,9,11,57.0,108.9,7,80.0,36.6,484,-6.0


### Describe

In [17]:
data.describe()

,race_track_id,kai,nichi,race_no,grade_id,num_of_horses,result,bracket_number,horse_number,age,weight_to_carry,time,ninki,win_odds,final_3_furlong,horse_weight,gain_and_loss_weight
count,5041.0,5041.000000,5041.000000,5041.000000,5041.000000,5041.000000,5032.000000,5041.000000,5041.000000,5041.000000,5041.000000,5032.000000,5041.000000,5041.000000,5032.000000,5041.000000,5039.000000
mean,50.0,26.969252,46.228923,4.614164,20.442968,14.054553,7.514706,4.804007,7.527276,3.415195,54.740230,109.342428,7.527276,74.707181,35.058963,465.397739,0.404247
std,0.0,14.339964,24.910084,2.726507,10.894933,2.911526,4.375827,2.285755,4.382516,1.310670,1.390913,2.264415,4.382516,108.165675,1.345381,29.059147,5.858690
min,50.0,10.000000,0.000000,0.000000,2.000000,6.000000,1.000000,1.000000,1.000000,2.000000,49.000000,104.300000,1.000000,1.100000,32.300000,368.000000,-34.000000
25%,50.0,10.000000,21.000000,2.000000,9.000000,12.000000,4.000000,3.000000,4.000000,3.000000,54.000000,107.800000,4.000000,7.900000,34.200000,444.000000,-2.000000
50%,50.0,21.000000,41.000000,5.000000,30.000000,15.000000,7.000000,5.000000,7.000000,3.000000,55.000000,109.100000,7.000000,26.100000,34.800000,466.000000,0.000000
75%,50.0,40.000000,70.000000,6.000000,30.000000,16.000000,11.000000,7.000000,11.000000,4.000000,56.000000,110.600000,11.000000,94.600000,35.600000,486.000000,2.000000
max,50.0,50.000000,91.000000,9.000000,31.000000,18.000000,18.000000,8.000000,18.000000,11.000000,59.000000,127.700000,18.000000,929.400000,48.100000,566.000000,34.000000


### Standardization

In [18]:
data_st = data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = scaler.fit_transform(data_st)
data_st = pd.DataFrame(df_scaled, columns=data_st.columns)
data_st.head()

,race_track_id,kai,nichi,race_no,grade_id,num_of_horses,result,bracket_number,horse_number,age,weight_to_carry,time,ninki,win_odds,final_3_furlong,horse_weight,gain_and_loss_weight
0,0.0,-1.183471,-0.651565,0.508333,0.969082,-0.362235,-1.488942,0.523290,0.107877,-0.316812,0.905804,-0.769559,-1.261336,-0.661155,-0.118166,-0.461096,-0.069006
1,0.0,-1.183471,-0.651565,0.508333,0.969082,-0.362235,-1.260391,1.398361,1.248887,-0.316812,0.905804,-0.592896,-1.033134,-0.643588,0.030505,0.433719,-0.069006
2,0.0,-1.183471,-0.651565,0.508333,0.969082,-0.362235,-1.031840,-0.789317,-1.033134,-0.316812,0.905804,-0.504564,-0.804932,-0.638040,-0.118166,0.709047,-0.069006
3,0.0,-1.183471,-0.651565,0.508333,0.969082,-0.362235,-1.031840,0.085754,-0.348528,-0.316812,0.905804,-0.504564,-1.489538,-0.665778,-0.043830,0.846711,-0.069006
4,0.0,-1.183471,-0.651565,0.508333,0.969082,-0.362235,-0.574738,-0.351782,-0.804932,-0.316812,-0.532243,-0.327900,0.792483,0.518634,0.104841,-0.461096,-0.069006


### Correlation

In [19]:
corr_matrix = data.corr()
corr_y = pd.DataFrame({"features":data.columns, "corr_y":corr_matrix["result"]}, index=None)
corr_y = corr_y.reset_index(drop=True)
corr_y.style.background_gradient()

,features,corr_y
0,race_track_id,nan
1,kai,-0.078411
2,nichi,-0.017852
3,race_no,-0.044456
4,grade_id,0.108027
5,num_of_horses,0.331989
6,result,1.000000
7,bracket_number,0.030716
8,horse_number,0.160173
9,age,0.047859


In [20]:
data.corr()

,race_track_id,kai,nichi,race_no,grade_id,num_of_horses,result,bracket_number,horse_number,age,weight_to_carry,time,ninki,win_odds,final_3_furlong,horse_weight,gain_and_loss_weight
race_track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
kai,NaN,1.000000,0.151430,-0.228419,0.020595,-0.237996,-0.078411,0.038545,-0.079056,-0.243307,-0.089397,0.103206,-0.079056,-0.024949,0.001994,0.060722,0.012276
nichi,NaN,0.151430,1.000000,-0.071234,-0.002993,-0.053459,-0.017852,-0.001554,-0.017758,-0.062614,-0.091435,0.137310,-0.017758,-0.033367,0.085944,-0.006051,0.029378
race_no,NaN,-0.228419,-0.071234,1.000000,0.027717,-0.135042,-0.044456,-0.001280,-0.044858,0.010469,0.029835,0.039336,-0.044858,-0.023138,-0.043295,-0.022837,0.025403
grade_id,NaN,0.020595,-0.002993,0.027717,1.000000,0.330394,0.108027,-0.031370,0.109749,-0.623794,-0.262511,0.527136,0.109749,0.168581,0.288315,-0.199261,-0.040507
num_of_horses,NaN,-0.237996,-0.053459,-0.135042,0.330394,1.000000,0.331989,-0.047497,0.332175,-0.063130,-0.186673,0.080374,0.332175,0.183730,0.153121,-0.138767,-0.009535
result,NaN,-0.078411,-0.017852,-0.044456,0.108027,0.331989,1.000000,0.030716,0.160173,0.047859,-0.135278,0.355406,0.651365,0.524571,0.509368,-0.112877,-0.004298
bracket_number,NaN,0.038545,-0.001554,-0.001280,-0.031370,-0.047497,0.030716,1.000000,0.892597,0.013459,0.015312,0.004374,0.018835,0.030195,-0.000900,-0.002536,0.005945
horse_number,NaN,-0.079056,-0.017758,-0.044858,0.109749,0.332175,0.160173,0.892597,1.000000,-0.020262,-0.057337,0.046948,0.142752,0.102790,0.064353,-0.057242,0.005181
age,NaN,-0.243307,-0.062614,0.010469,-0.623794,-0.063130,0.047859,0.013459,-0.020262,1.000000,0.349334,-0.456724,0.098149,-0.030393,-0.140111,0.195087,0.003053


### Model

In [21]:
from sklearn.model_selection import train_test_split
train_X = data_st[["bracket_number", "horse_number", "weight_to_carry", "win_odds", "horse_weight"]]
train_y = data_st["result"]
(train_X, test_X, train_y, test_y) = train_test_split(train_X, train_y, test_size=0.25, random_state=666)

### 重回帰

In [22]:
from sklearn.linear_model import LinearRegression
model_LR = LinearRegression()
model_LR.fit(train_X, train_y)
LR_y_test_pred = model_LR.predict(test_X)
LR_y_train_pred = model_LR.predict(train_X)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

### 予測精度算出

In [ ]:
from sklearn.metrics import mean_squared_error
print("LR RMSE train data: ", np.sqrt(mean_squared_error(train_y, LR_y_train_pred)))
print("LR RMSE train data: ", np.sqrt(mean_squared_error(test_y, LR_y_test_pred)))

NameError: name 'train_y' is not defined

In [ ]:
from sklearn.metrics import r2_score
print("LR r^2 train data: ", r2_score(train_y, LR_y_train_pred))
print("LR r^2 train data: ", r2_score(test_y, LR_y_test_pred))

In [ ]:
from sklearn.metrics import mean_absolute_error
print("LR MAE train data: ", mean_absolute_error(train_y, LR_y_train_pred))
print("LR MAE train data: ", mean_absolute_error(test_y, LR_y_test_pred))

In [ ]:
flg, ax = plt.subplots(figsize=(10, 6.0))
plt.xticks(rotation=45, fontsize=9)

plt.scatter(train_y, LR_y_train_pred)
plt.plot(train_y, train_y, "red")

plt.legend(fontsize=14)

In [ ]:
flg, ax = plt.subplots(figsize=(10, 6.0))
plt.xticks(rotation=45, fontsize=9)

plt.scatter(test_y, LR_y_test_pred)
plt.plot(test_y, test_y, "red")

plt.legend(fontsize=14)